# 0. Environment Setting

In [43]:
import pandas as pd
import numpy as np
import time
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
start_time = time.time()

In [45]:
# 학습완료된 모델, 전처리 완료된 데이터 다운로드
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-8A3zjgyBi3vjlWi9DX61IFxunCihb3-' -O demo_model.h5
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ApkVRmC0B2DgLjYQ3l1sbF6bx2GyBttY' -O demo_sentiment.xlsx

--2021-08-12 03:00:25--  https://docs.google.com/uc?export=download&id=1-8A3zjgyBi3vjlWi9DX61IFxunCihb3-
Resolving docs.google.com (docs.google.com)... 108.177.13.100, 108.177.13.139, 108.177.13.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.13.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-98-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bqfdl0u6qal2sfcnuiqarlfn5uj60443/1628737200000/15913661007502099898/*/1-8A3zjgyBi3vjlWi9DX61IFxunCihb3-?e=download [following]
--2021-08-12 03:00:25--  https://doc-04-98-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bqfdl0u6qal2sfcnuiqarlfn5uj60443/1628737200000/15913661007502099898/*/1-8A3zjgyBi3vjlWi9DX61IFxunCihb3-?e=download
Resolving doc-04-98-docs.googleusercontent.com (doc-04-98-docs.googleusercontent.com)... 108.177.12.132, 2607:f8b0:400c:c08::84
Connecting to doc-04-98-docs.googleusercontent.com (doc-04-98-d

In [46]:
# Mecab 설치용 스크립트 (약 3분 걸림)
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ hash mecab
+ echo 'mecab-ko is already installed'
mecab-ko is already installed
+ [[ -d /usr/local/lib/mecab/dic/mecab-ko-dic ]]
+ echo 'mecab-ko-dic is already installed'
mecab-ko-dic is already installed
++ python3 -c 'import pkgutil; print(1 if pkgutil.find_loader("MeCab") else 0)'
+ [[ 1 == \1 ]]
+ echo 'mecab-python is already installed'
mecab-python is already installed
+ echo Done.
Done.


In [47]:
# 잘 설치되었는지 테스트
from konlpy.tag import Mecab
m = Mecab()
m.pos('아버지가방에들어가신다.')

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('신다', 'EP+EF'),
 ('.', 'SF')]

# 1. Data loading

In [48]:
MAX_LEN = 50
VOCAB_SIZE = 13648

In [49]:
# Tokenizer 인코딩을 위한 학습용 엑셀 파일읽어들이기 약 30초 걸림.
df1 = pd.read_excel('./demo_sentiment.xlsx', sheet_name='Train')

In [50]:
def tokenize(s):
  return ['/'.join(t) for t in m.pos(s)]

train_docs = [tokenize(row[1]['sentence']) for row in df1.iterrows()]

In [51]:
tokenizer = Tokenizer(num_words = VOCAB_SIZE)
tokenizer.fit_on_texts(train_docs)

In [52]:
# 학습을 완료한 모델을 불러오기
loaded_model = load_model('demo_model.h5')

In [53]:
end_time = time.time()
print("Elapsed time for ready :", time.strftime('%H:%M:%S', time.localtime(end_time - start_time)))

Elapsed time for ready : 00:01:11


# 2. Ready to test

In [54]:
# 이용자가 알아보기 쉽게 결과를 분석해서 출력해 주는 함수들.
def sentiment_predict(sentence, log=False):
  sentences = tokenize(sentence)
  encoded = tokenizer.texts_to_sequences([sentences])
  padded = pad_sequences(encoded, maxlen=MAX_LEN)
  score = loaded_model.predict(padded)
  if log:
    print('Predict score: ', score)
  return score

def print_sentiment(lst):
  sent = ['기쁨', '불안', '당황', '슬픔', '분노', '상처']
  return sent[np.argmax(lst)]

In [55]:
print_sentiment(sentiment_predict('니가 어떻게 나에게 이럴수가 있어?', True))

Predict score:  [[0.64731556 0.10514342 0.11939979 0.07021657 0.0094684  0.04845627]]


'기쁨'

In [56]:
print_sentiment(sentiment_predict('아니, 니가 어떻게 나에게 이럴수가 있어?', True))

Predict score:  [[0.144451   0.05985985 0.43685517 0.2238917  0.03347811 0.10146421]]


'당황'

In [57]:
print_sentiment(sentiment_predict('감히 니가 어떻게 나에게 이럴수가 있어?', True))

Predict score:  [[0.10038799 0.00701625 0.0068602  0.04465635 0.8361947  0.00488456]]


'분노'

In [58]:
print_sentiment(sentiment_predict('그래, 너 참 잘났다.', True))

Predict score:  [[0.27811512 0.01648478 0.4914975  0.12529652 0.05300648 0.03559963]]


'당황'

In [59]:
print_sentiment(sentiment_predict('너 참 잘났다.', True))

Predict score:  [[0.48111573 0.03345519 0.33038217 0.05245472 0.07973    0.02286207]]


'기쁨'

In [60]:
print_sentiment(sentiment_predict('잘들논다.', True))

Predict score:  [[0.6240433  0.13965438 0.08255927 0.06877399 0.04602019 0.03894896]]


'기쁨'

In [61]:
print_sentiment(sentiment_predict('아이고, 잘들논다.', True))

Predict score:  [[0.08556036 0.08469302 0.04617947 0.6354081  0.1436197  0.00453937]]


'슬픔'

In [62]:
print_sentiment(sentiment_predict('방금 저녁을 먹었는데 왜 또 배가고프지?', True))

Predict score:  [[0.00149013 0.01824094 0.12491249 0.47754213 0.28039864 0.09741567]]


'슬픔'

In [63]:
print_sentiment(sentiment_predict('오늘 저녁에 내가 좋아하는 친구의 생일파티에 초대받았어', True))

Predict score:  [[0.80365133 0.02223701 0.01515409 0.11137892 0.00547363 0.0421051 ]]


'기쁨'

In [64]:
print_sentiment(sentiment_predict('이번 여름은 코로나 때문에 여행을 가지 못했어.', True))

Predict score:  [[0.13640569 0.15130658 0.23596878 0.04992492 0.41861936 0.00777466]]


'분노'

In [65]:
print_sentiment(sentiment_predict('다음 주에 개학인데, 나는 아직 방학숙제를 다 못했어.', True))

Predict score:  [[2.5222797e-04 5.8436580e-03 7.8574214e-03 9.6365899e-01 2.1302477e-02
  1.0852749e-03]]


'슬픔'

In [66]:
# 자유롭게 문장을 입력해보자.
messages = ['어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어.',
            '어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어. 그런데 나는 일본사람이야.',
            '어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어. 그런데 나는 한국사람이야.',
            '어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어. 그런데 나는 중국사람이야.',
            '하루종일 아무것도 먹지 않았더니 몸에 힘이 없어.',
            '신규확진 1823명, 열흘만에 다시 1800명 역대 세 번째 큰 규모',
            '오늘 날씨가 너무 더워서 공부하기 힘들어',
            '니가 어떻게 나에게 이럴 수가 있어?',
            '오늘 낮에 내가 황당한 스팸 문자를 받았어',
            '아니 왜 가만히 있는 우리를 괴롭히는거야?',
            '약속시간에 늦었어, 어떡하지?',
            # 아주 짧은 문장은 어떻게 될지
            '이게 되네?',
            # 엄청 긴 문장은 과연...
            '지난 7월 소비자물가 상승률이 연중 최고치인 2.6%인 것으로 확인되면서 하반기부터는 물가가 2% 이내로 안정화될 것이라는 정부 전망은 빗나갔다. 한은이 제시한 물가안정목표치(2.0%)도 훌쩍 넘길 가능성이 커지고 있다. 물가 불안이 확대되자 정부는 ‘물가 잡기’ 총력전을 펼치는 상황이다. 작황 부진과 폭염으로 인한 폐사 등으로 서민 체감이 큰 밥상 물가가 크게 오르고, 부동산 정책 실패로 집세는 천정부지인데다 수요회복으로 서비스 가격이 뛰는 등 “안오르는게 없는” 상황에 민심 역시 악화되고 있기 때문이다. 특히 지난 5월 조류독감(AI)가 종식됐는데도 가격이 내리지 않는 계란을 놓고는, 문재인 대통령이 직접 “달걀은 필수 먹거리인 만큼 소비자들에게도 피해가 갈 수 있으니 생산단계, 유통단계, 판매단계를 점검하라”고 부처들에 주문하기도 했다. 계란값 잡기에는 물가 총괄 부처인 기재부와 함께 농림부가 나섰고, 여기에 공정거래위원회까지 담합 가능성을 살펴보겠다며 가담한 상태다.'
]

for msg in messages:
  res = sentiment_predict(msg, True)
  print('입력문장: ', msg)
  print('감정상태: ', print_sentiment(res), '\n')

Predict score:  [[9.8391855e-01 1.0111522e-02 3.6681539e-04 4.6310546e-03 7.7225443e-04
  1.9985103e-04]]
입력문장:  어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어.
감정상태:  기쁨 

Predict score:  [[0.0016408  0.91486216 0.00112159 0.07445558 0.00658606 0.00133386]]
입력문장:  어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어. 그런데 나는 일본사람이야.
감정상태:  불안 

Predict score:  [[9.3392795e-01 4.4720571e-02 1.8743742e-03 1.6720915e-02 2.2889334e-03
  4.6740769e-04]]
입력문장:  어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어. 그런데 나는 한국사람이야.
감정상태:  기쁨 

Predict score:  [[9.8761320e-01 6.8793707e-03 1.1007789e-03 2.6576861e-03 1.5988504e-03
  1.5010379e-04]]
입력문장:  어제 한국과 일본이 배구경기를 했는데, 한국이 이겼어. 그런데 나는 중국사람이야.
감정상태:  기쁨 

Predict score:  [[0.00238074 0.15599112 0.2536087  0.450239   0.01611555 0.12166484]]
입력문장:  하루종일 아무것도 먹지 않았더니 몸에 힘이 없어.
감정상태:  슬픔 

Predict score:  [[0.3290947  0.04057013 0.44995427 0.16539021 0.00722547 0.00776529]]
입력문장:  신규확진 1823명, 열흘만에 다시 1800명 역대 세 번째 큰 규모
감정상태:  당황 

Predict score:  [[0.11999366 0.5234851  0.03501412 0.15736395 0.04501909 0